Create a qb:Slice for each CDID

In [1]:
import pandas as pd
import requests
from pathlib import Path
from io import BytesIO
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from IPython.display import display, HTML

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

csdb_classification_tables_url = 'https://drive.google.com/uc?export=download&id=1miAzQ6s8om4Ark3BpRk3Y90OAWfWErTb'
csdb_classification_table = BytesIO(session.get(csdb_classification_tables_url).content)
csdb_sheets = pd.read_excel(csdb_classification_table, sheet_name=None, index_col=0,
                            na_values=[], keep_default_na=False,
                            dtype={'COMMODITY': str, 'AREA': str, 'DIRECTION': str, 'BASIS': str,
                                   'PRICE': str, 'SEASADJ': str, 'PERIOD': str, 'PRODUCT': str,
                                   'COUNTRY': str})
for (sheet, df) in csdb_sheets.items():
    display(HTML('<b>' + sheet + '</b>'))
    display(df.head())

,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
cdid,,,,,,,
SDSX,2plus4,EU,BAL,BOP,CP,NSA,Q
SGLO,5minus8,EU,BAL,BOP,CP,NSA,Q
SESL,5plus6,EU,BAL,BOP,CP,NSA,Q
SFJC,7plus8,EU,BAL,BOP,CP,NSA,Q
LKTX,TminusO,EU,BAL,BOP,CP,NSA,Q


,PRODUCT,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
cdid,,,,,,,
P42L,24.2,EU,EX,BOP,CP,NSA,Q
P483,24.2,EU,EX,BOP,CP,SA,Q
P4DJ,24.2,EU,EX,BOP,CVM,NSA,Q
P4IZ,24.2,EU,EX,BOP,CVM,SA,Q
P3EP,24.2,EU,IM,BOP,CP,NSA,Q


,COUNTRY,DIRECTION,BASIS,SEASADJ,PERIOD
cdid,,,,,
KN2O,XS,BAL,BOP,NSA,Q
LGDS,V4,BAL,BOP,NSA,Q
L87P,V3,BAL,BOP,NSA,Q
L87J,V2,BAL,BOP,NSA,Q
MHN8,I7,BAL,BOP,NSA,Q


In [2]:
codelist_url = 'https://drive.google.com/uc?export=download&id=161OtInylx2518gmhRu7UgUYnZZ_x9FQr'
codelist = pd.read_csv(BytesIO(session.get(codelist_url).content), index_col=0,
                       na_values=[], keep_default_na=False,
                       dtype={'AREA': str, 'DIRECTION': str, 'BASIS': str,
                              'PRICE': str, 'SEASADJ': str, 'PERIOD': str},
                       converters={'COMMODITY': lambda x: str(x).strip()})
codelist

,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
cdid,,,,,,,
AJFB,Canadian dollar,UK,,BE,CP,NSA,
AJFD,Swiss franc,UK,,BE,CP,NSA,
AJFI,Swedish kroner,UK,,BE,CP,NSA,
AJFJ,Norwegian kroner,UK,,BE,CP,NSA,
AJFK,Danish kroner,UK,,BE,CP,NSA,
AJFO,Japanese yen,UK,,BE,CP,NSA,
AJFP,Australian Dollar,UK,,BE,CP,NSA,
AJFU,Hong Kong Dollar,UK,,BE,CP,NSA,
AJFV,New Zealand Dollar,UK,,BE,CP,NSA,


Some CDIDs aren't listed in the above, so we need to grab all those referenced.

In [3]:
query = '''SELECT DISTINCT ?cdid
WHERE {
  [] <http://gss-data.org.uk/def/attribute/cdid> ?cdid .
}'''

endpoint = 'https://production-drafter-ons-alpha.publishmydata.com/v1/sparql/live'

cdids = pd.read_csv(BytesIO(requests.get(endpoint,
                                         headers={'Accept': 'text/csv'},
                                         params={'query': query}).content))
prefixlen = len('http://gss-data.org.uk/def/cdid/')
cdids.cdid = cdids.cdid.map(lambda x: x[prefixlen:])
cdids

,cdid
0,APQL
1,FAPO
2,FDSG
3,FDTF
4,FDYI
5,FDYQ
6,FEBA
7,FEHH
8,FGXJ
9,FGZA


In [4]:
sheets_cdids = pd.concat(list(csdb_sheets.values()) + [codelist])
defined_cdids = set(sheets_cdids.index.values)
extra_cdids = set(cdids.cdid.unique()).difference(defined_cdids)
len(extra_cdids)

/home/alex/.virtualenvs/databaker/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


416

At least need something like
```
<http://gss-data.org.uk/def/cdid/APQL>
  a qb:Slice;
  skos:notation "APQL";
  rdfs:label "APQL";
  .
```

In [5]:
from rdflib import Graph, Literal, BNode, Namespace, RDF, URIRef, RDFS, OWL, XSD
from rdflib.namespace import SKOS
from rdflib.collection import Collection

CDID = Namespace('http://gss-data.org.uk/def/cdid/')
QB = Namespace('http://purl.org/linked-data/cube#')

g = Graph()
g.bind('skos', SKOS)
g.bind('rdfs', RDFS)
g.bind('cdid', CDID)
g.bind('qb', QB)

for cdid in defined_cdids.union(extra_cdids):
    term = CDID.term(cdid)
    g.add((term, RDF.type, QB.Slice))
    g.add((term, SKOS.notation, Literal(cdid)))
    g.add((term, RDFS.label, Literal(cdid)))

print(g.serialize(format='n3').decode('utf-8')[:600])

@prefix cdid: <http://gss-data.org.uk/def/cdid/> .
@prefix qb: <http://purl.org/linked-data/cube#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix skos: <http://www.w3.org/2004/02/skos/core#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

cdid:AJFB a qb:Slice ;
    rdfs:label "AJFB" ;
    skos:notation "AJFB" .

cdid:AJFD a qb:Slice ;
    rdfs:label "AJFD" ;
    skos:notation "AJFD" .

cdid:AJFI a qb:Slice ;
    rdfs:label "AJFI" ;
    skos:notation "AJFI"


In [6]:
out = Path('out')
out.mkdir(exist_ok=True, parents=True)
with open(out / 'cdids.ttl', 'wb') as f:
    g.serialize(f, format='n3')